In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Dropout, LSTM, BatchNormalization, Input, Reshape, TimeDistributed, GlobalAveragePooling1D
)
#from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
# Constants
IMAGE_SIZE = (224, 224)
PATCH_SIZE = (16, 16) 
PATCHES_PER_IMAGE = (IMAGE_SIZE[0] // PATCH_SIZE[0]) ** 2
EPOCHS = 30
BATCH_SIZE = 32

In [ ]:
# Function to load and preprocess fingerprint images
def load_images_with_labels(live_paths, fake_paths):
    images, labels = [], []
    for paths, label in zip([live_paths, fake_paths], [0, 1]):
        for path in paths:
            for subdir, _, files in os.walk(path):
                for file in files:
                    if file.lower().endswith(('.png', '.jpeg', '.jpg', '.tif', '.bmp')):
                        img_path = os.path.join(subdir, file)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_resized = cv2.resize(img, IMAGE_SIZE)
                            images.append(img_resized)
                            labels.append(label)
    return np.array(images), np.array(labels)


In [ ]:
# Function to extract patches from images
def extract_patches(images):
    patchified = []
    for img in images:
        patches = []
        for i in range(0, IMAGE_SIZE[0], PATCH_SIZE[0]):
            for j in range(0, IMAGE_SIZE[1], PATCH_SIZE[1]):
                patch = img[i:i+PATCH_SIZE[0], j:j+PATCH_SIZE[1], :]
                patches.append(patch)
        patchified.append(patches)
    return np.array(patchified)

# 
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.layers import Bidirectional, GRU

def build_convnext_bigru(input_shape_patch=(56, 56, 3), patch_count=PATCHES_PER_IMAGE):
    patch_input = Input(shape=(patch_count,) + input_shape_patch)
    
    # Feature extractor with ConvNeXt-Tiny
    base_model = ConvNeXtTiny(include_top=False, weights='imagenet', pooling='avg', input_shape=input_shape_patch)
    base_model.trainable = False  # Freeze to use as feature extractor

    # Apply ConvNeXt to each patch
    x = TimeDistributed(base_model)(patch_input)
    
    # BiGRU 
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = GlobalAveragePooling1D()(x)

    # Fully connected layers
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.3)(x)

    output = Dense(1, activation='sigmoid')(x)
    return Model(inputs=patch_input, outputs=output)
# Provide your actual dataset paths here
live_paths = [

    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Testing/SwipeTest/SwipeTest/Live",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Training/SwipeTest/SwipeTest/Live"
]
fake_paths = [
"/livedetfinerprin/Fingerprint/2013/Fingerprint/Testing/SwipeTest/SwipeTest/Spoof/BodyDouble",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Testing/SwipeTest/SwipeTest/Spoof/Latex",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Testing/SwipeTest/SwipeTest/Spoof/PlayDoh",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Testing/SwipeTest/SwipeTest/Spoof/WoodGlue",

"/livedetfinerprin/Fingerprint/2013/Fingerprint/Training/SwipeTest/SwipeTest/Spoof/BodyDouble",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Training/SwipeTest/SwipeTest/Spoof/Latex",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Training/SwipeTest/SwipeTest/Spoof/PlayDoh",
    "/livedetfinerprin/Fingerprint/2013/Fingerprint/Training/SwipeTest/SwipeTest/Spoof/WoodGlue",    
]



In [ ]:
# Load and preprocess data
images, labels = load_images_with_labels(live_paths, fake_paths)
images = preprocess_input(images.astype('float32'))
patches = extract_patches(images)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    patches, labels, test_size=0.2, stratify=labels, random_state=42
)

# Build and train model
model = build_convnext_bigru()
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    callbacks=[early_stop, lr_reduce]
)


In [ ]:
# Predictions and evaluation
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
# Print classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=["Live", "Fake"]))
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)


In [ ]:
print(f"Accuracy: {accuracy:.4f}\nAUC: {auc:.4f}\nF1: {f1:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}")
# Updated Plotting Functions
def plot_training_accuracy(history):
    plt.figure(figsize=(10, 7))
    plt.plot(history.history['accuracy'], label='Train Accuracy', color='#ff7f0e', linestyle='--', linewidth=2, marker='o')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='#1f77b4', linestyle='-', linewidth=2, marker='s')
    plt.title('Training and Validation Accuracy', fontsize=24, fontweight='bold')
    plt.xlabel('Epochs', fontsize=22, fontweight='bold')
    plt.ylabel('Accuracy', fontsize=22, fontweight='bold')
    plt.xticks(fontsize=20, fontweight='bold')
    plt.yticks(fontsize=20, fontweight='bold')
    plt.legend(loc='lower right', fontsize=20)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

In [ ]:
def plot_training_loss(history):
    plt.figure(figsize=(10, 7))
    plt.plot(history.history['loss'], label='Train Loss', color='#d62728', linestyle='--', linewidth=2, marker='x')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='#9467bd', linestyle='-', linewidth=2, marker='d')
    plt.title('Training and Validation Loss', fontsize=24, fontweight='bold')
    plt.xlabel('Epochs', fontsize=22, fontweight='bold')
    plt.ylabel('Loss', fontsize=22, fontweight='bold')
    plt.xticks(fontsize=20, fontweight='bold')
    plt.yticks(fontsize=20, fontweight='bold')
    plt.legend(loc='upper right', fontsize=20)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
def plot_precision_recall_f1(precision, recall, f1):
    metrics = ['Precision', 'Recall', 'F1 Score']
    values = [precision, recall, f1]
    colors = ['#2ca02c', '#ff9896', '#98df8a']  # Greenish tones for contrast
    plt.figure(figsize=(10, 7))
    plt.bar(metrics, values, color=colors, edgecolor='black', linewidth=1.5)
    plt.title('Precision, Recall, and F1 Score', fontsize=24, fontweight='bold')
    plt.xlabel('Metrics', fontsize=22, fontweight='bold')
    plt.ylabel('Scores', fontsize=22, fontweight='bold')
    plt.xticks(fontsize=20, fontweight='bold')
    plt.yticks(fontsize=20, fontweight='bold')
    plt.ylim(0, 1.1)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
def plot_confusion_matrix_percentage(y_test, y_pred):
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix_percentage, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, annot_kws={"size": 16})
    plt.title('Confusion Matrix (Percentage)', fontsize=24, fontweight='bold')
    plt.xlabel('Predicted Class', fontsize=22, fontweight='bold')
    plt.ylabel('True Class', fontsize=22, fontweight='bold')
    plt.xticks(fontsize=20, fontweight='bold')
    plt.yticks(fontsize=20, fontweight='bold')
    plt.show()


In [ ]:
def plot_roc_curve(y_test, y_pred_prob):
    from sklearn.metrics import roc_curve, auc  # Ensure proper import for auc
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc_score_value = auc(fpr, tpr)  # Renamed variable to avoid conflict

    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score_value:.4f}', linewidth=2, color='#17becf', linestyle='-')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=1, alpha=0.7)
    plt.title('ROC Curve', fontsize=24, fontweight='bold')
    plt.xlabel('False Positive Rate', fontsize=22, fontweight='bold')
    plt.ylabel('True Positive Rate', fontsize=22, fontweight='bold')
    plt.legend(loc='lower right', fontsize=20)
    plt.xticks(fontsize=20, fontweight='bold')
    plt.yticks(fontsize=20, fontweight='bold')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
# Plot Results
plot_training_accuracy(history)
plot_training_loss(history)
plot_precision_recall_f1(precision, recall, f1)
plot_confusion_matrix_percentage(y_test, y_pred)
plot_roc_curve(y_test, y_pred_prob)
